In [ ]:
s1 = lambda t : 0.05 if t <= 0.25 else (0.65 if t <= 0.5 else (0.15 if t <= 0.75 else 0.35))
s2 = lambda t : 0.775 if t <= 0.25 else (-0.025 if t <= 0.5 else (0.975 if t <= 0.75 else 0.275))

g1 = getSourceTerm(params.x1, params)
g2 = getSourceTerm(params.x2, params)

K_u, u = solveStateEquation([g1, g2], [s1, s2], params)
grid_size = K_u[0].x.array.shape

# Creating the measured state
raw_matrices = [K_u_t.x.array for K_u_t in K_u]
norms = [np.linalg.norm(matrix, ord=2) for matrix in raw_matrices]
scale = params.randomFactor * np.mean(norms) / grid_size[0] * params.area
np.random.seed(42)
yd = [fem.Function(params.V) for _ in K_u]
for idx in range(len(yd)):
    yd[idx].x.array[:] = K_u[idx].x.array + np.random.normal(loc=0.0, scale=scale, size=grid_size)
params.yd = yd
phi = [fem.Function(params.V) for _ in K_u]
for idx in range(len(phi)):
    phi[idx].x.array[:] = 0 * K_u[idx].x.array - yd[idx].x.array
    
adjointState = solveAdjointEquation(phi, params)
#timeDependentVariableToGif(params.yd, filename="observation_state.gif", varname="state", slowMoFactor=2, T=params.T)
#timeDependentVariableToGif(yd, filename="adjoint.gif", varname="adjoint", slowMoFactor=2, T=T)

adjointValues = np.ndarray((len(adjointState), params.d), dtype=np.float64)
for idx, func in enumerate(adjointState):
    adjointValues[idx] = getValueOfFunction(params.V, func, [params.x1, params.x2])[:, 0]

first_dual = np.zeros((len(adjointState), params.d), dtype=np.float64)
first_dual[0] = adjointValues[0]
for idx in range(1, len(adjointState)):
    first_dual[idx] = first_dual[idx - 1] + adjointValues[idx]
first_dual /= (len(adjointState) - 1)

second_dual = np.zeros((len(adjointState), params.d), dtype=np.float64)
second_dual[0] = first_dual[0]
for idx in range(1, len(adjointState)):
    second_dual[idx] = second_dual[idx - 1] + first_dual[idx]
second_dual /= (len(adjointState) - 1)

plot_array(first_dual, params.T)
plot_array(second_dual, params.T)

idx_xS = np.argmax(np.linalg.norm(first_dual, axis=1))
idx_xK = np.argmax(np.linalg.norm(second_dual, axis=1))

sigma_xS = first_dual[idx_xS] / np.linalg.norm(first_dual[idx_xS])
sigma_xK = second_dual[idx_xK] / np.linalg.norm(second_dual[idx_xK])